### Part 2. Writing to Database ###
1. Pandas
2. Using SQL Queries
3. Using ORM

In [1]:
# import dependencies
import sqlalchemy as sql
import csv

In [3]:
# connect to db
db_url='db/database.sqlite'
connection_url=f'sqlite:///{db_url}'

engine=sql.create_engine(connection_url)

#### Option 1. Pandas ####

In [2]:
import pandas as pd

df=pd.read_csv('data/data.csv')
df.head()

/Users/kevinlee/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


,title,open_date,movie_link,poster_link
0,The Thorn,"Opened Mar 06, 2023",/m/the_thorn_2023,https://resizing.flixster.com/bFeiUQwfc2drDpWu...
1,Tu Jhoothi Main Makkaar,"Opens Mar 08, 2023",/m/tu_jhoothi_main_makkaar,https://resizing.flixster.com/fQ4D1iiRPiwNdFXG...
2,Mitran Da Naa Chalda,"Opens Mar 08, 2023",/m/mitran_da_naa_chalda,https://resizing.flixster.com/0q8o60vxY3-QnvPy...
3,Uchiyan Ne Gallan Tere Yaar Diyan,"Opens Mar 08, 2023",/m/uchiyan_ne_gallan_tere_yaar_diyan,https://resizing.flixster.com/CHvICSdOHXUD-cfd...
4,Scream VI,"Opens Mar 10, 2023",/m/scream_vi,https://resizing.flixster.com/SNmo199mLTKkD3lO...


In [5]:
df.to_sql('movies', con=engine, index=False)

In [8]:
# test
sql_query='''
SELECT * 
FROM movies;
'''

engine.execute(sql_query).fetchall()

OperationalError: (sqlite3.OperationalError) no such table: movies
[SQL: 
SELECT * 
FROM movies
]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [67]:
# delete records
sql_query='''
DROP TABLE movies;
'''

engine.execute(sql_query)

#### Option 2. Using SQL Queries ####
1. Define Table
2. Insert Records

In [29]:
# step 1
create_query='''
CREATE TABLE movies(
    title VARCHAR, 
    open_date VARCHAR, 
    movie_link VARCHAR, 
    poster_link VARCHAR
);
'''

engine.execute(create_query)

In [34]:
# check if table was created
engine.table_names()

[]

In [30]:
# step 2
insert_query='''
INSERT INTO movies
VALUES 
'''

In [31]:
with open('data/data.csv') as f: 
    csv_reader=csv.reader(f)
    header=next(csv_reader)
    for row in csv_reader: 
        insert_query+=str(tuple(row)) + ', '
        
insert_query=insert_query[:-2]+'; '

engine.execute(insert_query)

In [32]:
# test
sql_query='''
SELECT * 
FROM movies;
'''

engine.execute(sql_query).fetchall()

[('The Thorn', 'Opened Mar 06, 2023', '/m/the_thorn_2023', 'https://resizing.flixster.com/bFeiUQwfc2drDpWul3gV3g9PAXk=/180x258/v2/https://resizing.flixster.com/m9SUebSjlMIrn5G9VEGmx6mKL3c=/ems.cHJkLWVtcy1hc3NldHMvbW92aWVzL2ZmNmVhOThmLWZiN2EtNGMzOS1hODc5LThjNTI5YjU4OTc4Yy5qcGc='),
 ('Tu Jhoothi Main Makkaar', 'Opens Mar 08, 2023', '/m/tu_jhoothi_main_makkaar', 'https://resizing.flixster.com/fQ4D1iiRPiwNdFXGE6za19k-bhA=/180x258/v2/https://resizing.flixster.com/vqEGJotvD5G3tMkJUb8eBWKC_vk=/ems.cHJkLWVtcy1hc3NldHMvbW92aWVzL2UwZGFhNDY0LWJmZDUtNDRhZC04YzQ2LWEwNmIzN2MzZmQ3Zi5qcGc='),
 ('Mitran Da Naa Chalda', 'Opens Mar 08, 2023', '/m/mitran_da_naa_chalda', 'https://resizing.flixster.com/0q8o60vxY3-QnvPyzmu-p2e1-2U=/180x258/v2/https://resizing.flixster.com/zWOjQYmc6n88sUHCh8DFiIpcA4I=/ems.cHJkLWVtcy1hc3NldHMvbW92aWVzLzBlNDEwNTUwLTc5MjktNDJiMi1iNjI2LTY4Y2VjZDI3MWRlOS5qcGc='),
 ('Uchiyan Ne Gallan Tere Yaar Diyan', 'Opens Mar 08, 2023', '/m/uchiyan_ne_gallan_tere_yaar_diyan', 'https://resizing.

#### Option 3. Using ORM ####
* use `declarative_base` if schema/table doesn't exist or
* use `automap_base` if schema/table exist

In [68]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, String, Integer

from sqlalchemy.orm import Session

In [69]:
# use declarative_base to create a Base if schema doesn't exist
Base=declarative_base()

In [70]:
# define Python class to map to table
class Movie(Base): 
    __tablename__='movies'
    id=Column(Integer, primary_key=True)
    title=Column(String)
    open_date=Column(String)
    movie_link=Column(String)
    poster_link=Column(String)

In [71]:
# check db if schema already exists and/or matches with Base
Base.metadata.create_all(engine)

In [49]:
# use automap_base to create a Base if schema exists
# Base=automap_base()
# Base.prepare(engine, reflect=True)

# print(f'Tables list: {Base.classes.keys()}')

# Movie=Base.classes.movies

Tables list: ['movies']


In [72]:
session=Session(engine)

In [73]:
idx=1
with open('data/data.csv') as f: 
    csv_reader=csv.reader(f)
    header=next(csv_reader)
    for row in csv_reader: 
        print(f'Inserting {idx}...')
        # use dictionary comprehension to create movie dictionary
        movie_dict={k:v for (k, v) in zip(header, row)}
        # use movie dictionary to create new Movie objects
        movie_dict['id']=idx
        new_movie=Movie(**movie_dict)
        # insert record
        session.add(new_movie)
        idx+=1
        
session.commit()

Inserting 1...
Inserting 2...
Inserting 3...
Inserting 4...
Inserting 5...
Inserting 6...
Inserting 7...
Inserting 8...
Inserting 9...
Inserting 10...
Inserting 11...
Inserting 12...
Inserting 13...
Inserting 14...
Inserting 15...
Inserting 16...
Inserting 17...
Inserting 18...
Inserting 19...
Inserting 20...
Inserting 21...
Inserting 22...
Inserting 23...
Inserting 24...
Inserting 25...
Inserting 26...
Inserting 27...
Inserting 28...
Inserting 29...
Inserting 30...
Inserting 31...
Inserting 32...
Inserting 33...
Inserting 34...
Inserting 35...
Inserting 36...
Inserting 37...
Inserting 38...
Inserting 39...
Inserting 40...
Inserting 41...
Inserting 42...
Inserting 43...
Inserting 44...
Inserting 45...
Inserting 46...
Inserting 47...
Inserting 48...
Inserting 49...
Inserting 50...
Inserting 51...
Inserting 52...
Inserting 53...
Inserting 54...
Inserting 55...
Inserting 56...
Inserting 57...
Inserting 58...
Inserting 59...
Inserting 60...
Inserting 61...
Inserting 62...
Inserting 63...
I

In [66]:
engine.execute('select * from movies').fetchall()

[(1, 'The Thorn', 'Opened Mar 06, 2023', '/m/the_thorn_2023', 'https://resizing.flixster.com/bFeiUQwfc2drDpWul3gV3g9PAXk=/180x258/v2/https://resizing.flixster.com/m9SUebSjlMIrn5G9VEGmx6mKL3c=/ems.cHJkLWVtcy1hc3NldHMvbW92aWVzL2ZmNmVhOThmLWZiN2EtNGMzOS1hODc5LThjNTI5YjU4OTc4Yy5qcGc=')]